# Review Analyzer
### By Revanth Krishna Bachu

This code analyzes reviews to predict the rating the reviewer might have given, an analysis of the review, and a list of tags for the review. It will also compile the tags into a file and sort them from most to least occurring within all of the reviews.


**Things to keep in mind:**

The ChatGpt API can only take 3 requests per minute for non subscription members

This code uses the GPT-3.5 Turbo model. Pricing for all the models can be found at https://openai.com/pricing

First, run the following 3 cells. Then start running the cells for either the Single Review Analyzer or the Multiple Review Analyzer depending on which you need.

In [ ]:
!pip install openai

In [ ]:
import os
import openai
import time

In [ ]:
api_key = input("Enter your OpenAPI Key here:  ")
openai.api_key = api_key
time_between_messages = 60 / 3 #Non Subscription members have a limit of 3 requests per minute.

# Single Review Analyzer

You will be prompted for the review, and the predicted rating, analysis, and tags will be printed out

If you have a gpt 4 key, you can use this code to add an image to give the model more context

In [ ]:
import base64
import requests
user_input=input("Enter your review here:  ")


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = input("Enter your image path: ")

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4o-mini",
  "messages": [
      {"role": "system", "content": "You may only answer with a single number."},
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What rating between 0 and 5 inclusive do you think this reviewer would give based on their review and image: "+user_input+"/n Give only the number."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

In [ ]:
payload = {
  "model": "gpt-4o-mini",
  "messages": [
      {"role": "system", "content": "You may only answer with a list in the format of: -item1\n-item2"},
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Summarize the main points of this review into a list of generic tags: "+user_input
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

payload = {
  "model": "gpt-4o-mini",
  "messages": [
      {"role": "system", "content": "ou are a helpful assistant"},
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Give me a summary of feedback from this review: "+user_input
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

Use this code instead if you do not have access to gpt4

In [ ]:
user_input=input("Enter your review here:  ")

r = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You may only answer with a single number."}, 
        {"role": "user", "content": "What rating between 0 and 5 inclusive do you think this reviewer would give based on their review: "+user_input+"/n Give only the number."}
    ]
)
    
t = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You may only answer with a list in the format of: -item1\n-item2"}, 
        {"role": "user", "content": "Summarize the main points of this review into a list of generic tags: "+user_input}
    ]
)
    
f = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"}, 
        {"role": "user", "content": "Give me a summary of feedback from this review: "+user_input}
    ]
)
    
    #Calling the models for chatgpt's answer; each call must be staggered to work within the number of calls per minute
    
predicted_rating = r.choices[0].message.content
time.sleep(time_between_messages)
predicted_tags= t.choices[0].message.content
time.sleep(time_between_messages)
predicted_feedback= f.choices[0].message.content

print("\nPredicted Rating: ", predicted_rating)
print("\nAnalysis: ", predicted_feedback)
print("\nTags: ", predicted_tags)

# Multiple Reviews Analyzer

You will be prompted to enter in the file paths for the file with all your reviews (which must be spaced so that there is a completely blank line between each review), and the two files that are to be written to, one file that contains the predicted rating, analysis, and tags for each review, and the other file contains the tags from all the reviews ordered by frequency.

In [ ]:
file_path_reviews = input("Enter the path of the file with the reviews: ")                     #File with all the reviews, the reviews must be spaced out with a full blank line
file_path_analysis = input("Enter the path of the file that will hold the analysis: ")         #File where the full analysis will go
file_path_tags= input("Enter the path of the file that will hold the sorted tags:  ")          #File where the sorted tags will go

In [ ]:
#open the file of Reviews to read, then split them up into an array of strings
#Reviews in the reviews file should be seperated by a whole blank line
try:
    with open(file_path_reviews, "r") as file:
        content = file.read()
    reviews = content.split("\n\n")
    reviews = [review.strip() for review in reviews]
except:
    print("Error occured with file path: ", file_path_reviews)

In [ ]:
rating_predictions=[]
analysis=[]
tagging=[]
for review in reviews:
    
    #Initialization of the models with the review
    
    r = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You may only answer with a single number."}, 
        {"role": "user", "content": "What rating between 0 and 5 inclusive do you think this reviewer would give based on their review: "+review+"/n Give only the number."}
      ]
    )
    
    t = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You may only answer with a list in the format of: -item1\n-item2"}, 
        {"role": "user", "content": "Summarize the main points of this review into a list of generic tags: "+review}
      ]
    )
    
    f = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are a helpful assistant"}, 
        {"role": "user", "content": "Give me a summary of feedback from this review: "+review}
      ]
    )
    
    #Calling the models for chatgpt's answer; each call must be staggered to work within the number of calls per minute
    
    rating = r.choices[0].message.content
    time.sleep(time_between_messages)
    tags= t.choices[0].message.content
    time.sleep(time_between_messages)
    feedback= f.choices[0].message.content
    time.sleep(time_between_messages)
    
    #add the answers to an array of stings
    
    rating_predictions.append(rating)
    analysis.append(feedback)
    tagging.append(tags)

In [ ]:
#Opens a text file to write the overall analysis
try:
    with open(file_path_analysis, "w") as file:
        for r, c, a, t, in zip(reviews, rating_predictions, analysis, tagging):
            file.write("Review: "+r+"\n\n")
            file.write("Predicted Rating:\n "+c+"\n\n")
            file.write("Analysis: "+a+"\n\n")
            file.write("Predicted Tags: \n"+t+"\n\n")
except:
    print("Error occured with file path: ", file_path_analysis)

In [ ]:
# To order the tags and find their occurence, a dictionary is made and assigned all the induvidual tags from each review

d={}
for tags in tagging:
    li = tags.strip().split('\n')
    for tag in li:
        key = tag[1:]  #chatgpt uses hyphens at the start of every item in the list so we start from the second character
        if key in d:
            d[key] =d[key]+1
        else:
            d[key] = 1

#the "sorted" fucntion will sort the dictionary into descending order based on the amount of times the tag occured

sd=dict(sorted(d.items(), key=lambda item: item[1], reverse=True))

#the sorted tags are then written into another file, along with the number of times they occured
try:
    with open(file_path_tags, "w") as file:
        for key, value in sd.items():
            file.write(key+": "+str(value)+"\n\n")
except:
    print("Error occured with file path: ", file_path_tags)